# Tutorial 1: Environment Creation and Training

In this tutorial, we will show a simple example of how to create a gym environment using [EAGERx](https://eagerx.readthedocs.io/en/master/).
Also, we will use this environment to train a policy using [Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/).

The following will be covered:
- Creating a [Graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html) with an [Object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html)
- How to use this [Graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html) and a [Engine](https://eagerx.readthedocs.io/en/master/guide/api_reference/engine/index.html) to create an [Eagerx Environment](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html)
- How to train a policy with the [EAGERx Environment](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html)

In the remainder of this tutorial we will go more into detail on these concepts.


## Pendulum Swing-up

We will create an environment for solving the classic control problem of swinging up an underactuated pendulum, very similar to the [Pendulum-v1 environment](https://www.gymlibrary.ml/environments/classic_control/pendulum/).
Our goal is to swing up this pendulum to the upright position and keep it there, while minimizing the velocity of the pendulum and the input voltage.

Since the dynamics of a pendulum actuated by a DC motor are well known, we can simulate the pendulum by integrating the corresponding ordinary differential equations (ODEs):


$\mathbf{x} = \begin{bmatrix} \theta \\ \dot{\theta} \end{bmatrix} \\ \dot{\mathbf{x}} = \begin{bmatrix} \dot{\theta} \\ \frac{1}{J}(\frac{K}{R}u - mgl \sin{\theta} - b \dot{\theta} - \frac{K^2}{R}\dot{\theta})\end{bmatrix}$

with $\theta$ the angle w.r.t. upright position, $\dot{\theta}$ the angular velocity, $u$ the input voltage, $J$ the inertia, $m$ the mass, $g$ the gravitational constant, $l$ the length of the pendulum, $b$ the motor viscous friction constant, $K$ the motor constant and $R$ the electric resistance.

<img src="./figures/pendulum.GIF" width="480" />


## Activate GPU (Colab only)

When in Colab, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down


## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.

In [1]:
try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials >> /tmp/eagerx_install.txt 2>&1"}
if 'google.colab' in str(get_ipython()):
    !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
    !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()

Not running on CoLab.
Execute ROS commands as "!...".
ROS noetic available.


## Let's get started

First we will import EAGERx and initialize it.
As mentioned before, EAGERx makes use of ROS functionality for communication and during initialization a ROS master is started if there isn't one running already.

In [2]:
import eagerx
# Initialize eagerx (starts roscore if not already started.)
eagerx.initialize("eagerx_core")

... logging to /home/jelle/.ros/log/39a87530-dc2b-11ec-9aaf-cbc881d02458/roslaunch-jelle-Alienware-m15-R4-12087.log
started roslaunch server http://10.152.153.161:38351/
ros_comm version 1.15.14


SUMMARY

PARAMETERS
 * /rosdistro: noetic
 * /rosversion: 1.15.14

NODES

auto-starting new master
process[master]: started with pid [12140]
ROS_MASTER_URI=http://localhost:11311
setting /run_id to 39a87530-dc2b-11ec-9aaf-cbc881d02458
process[rosout-1]: started with pid [12165]
started core service [/rosout]


An `Object` is an entity that has inputs (sensors), outputs (actuators) and states (that can be reset at the beginning of an episode).

We are going to create one object (the pendulum). For this first tutorial, we don't want to go into details too much and start with existing objects.
Note that we import the pendulum.
While this might look like an unused import, it is not.
During the import, the pendulum object is registered and we can therefore make it based on its ID, i.e. *Pendulum*.

Before making the object, we will first obtain some info on the *Pendulum*, such that we know with what arguments we should make it.

In [3]:
import eagerx_tutorials.pendulum  # Registers Pendulum

eagerx.Object.info("Pendulum")

Registered entity_id=`Pendulum`:
   entity_type: `Object`
   module: `eagerx_tutorials.pendulum.objects`
   file: `/home/jelle/eagerx_dev/eagerx_tutorials/eagerx_tutorials/pendulum/objects.py`

Supported engines:
 - OdeEngine

Make this spec with (use `entity_id: str = "Pendulum"`):
   spec = Object.make(entity_id: str, name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None, render_fn: str = None)

class Pendulum(Object):
   spec(spec: eagerx.core.specs.ObjectSpec, name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None, render_fn: str = None):
      docs:
         Object spec of Pendulum

   agnostic(spec: eagerx.core.specs.ObjectSpec, rate: float):
      config:
       - render_shape: [480, 480]
       - render_fn: pendulum_render_fn
      sensors:
       - theta: <class 'std_msgs.msg._Float32.Float32'>
      

We see that the `eagerx.Object.info("Pendulum")` provides us information on the *Pendulum* object.
It has four sensors (*theta*, *dtheta*, *image*, *u_applied*), one actuator (*u*) and two states (*model_state*, *model_parameters*).
Here *theta*, *dtheta* and *u* correspond to $\theta$, $\dot{\theta}$ and $u$, respectively.
For now, we are only interested in how to make this object, other information will be covered in later tutorials.
We can make the *Pendulum* object with the `eagerx.Object.make` method with the required arguments *entity_id* and (a unique) *name*.
Furthermore, we will specify which actuators, sensors and states we will use:

In [4]:
# Make pendulum
pendulum = eagerx.Object.make("Pendulum", "pendulum", actuators=["u"], sensors=["theta", "dtheta", "image"], states=["model_state"])

Next, we create a [Graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html) and add the pendulum to it.

The graph describes the interconnection of nodes and objects.
In this way, the creation of an environment becomes modular.
This allows users to create an implementation for nodes and objects once, and easily create new environments by reusing these implementations.
Also, this allows to construct complex environments using nodes and objects as basic building blocks.

After adding the pendulum to the graph, we will connect the actuator *u* to a new action called *voltage*.
We will connect the sensors *theta* and *dtheta* to the observations *angle* and *angular_velocity*, respectively.
In this way, the agent will be able to send actions to control $u$ of the pendulum and observe $\theta$ and $\dot{\theta}$.

Finally, we will also render the *image* sensor in order to visualize the pendulum.
More detailed information on rendering is covered in another tutorial.

In [5]:
# Define rate (depends on rate of ode)
rate = 30.0

# Initialize empty graph
graph = eagerx.Graph.create()

# Add pendulum to the graph
graph.add(pendulum)

# Connect the pendulum to an action and observation
graph.connect(action="voltage", target=pendulum.actuators.u)
graph.connect(source=pendulum.sensors.theta, observation="angle")
graph.connect(source=pendulum.sensors.dtheta, observation="angular_velocity")

# Render image
graph.render(source=pendulum.sensors.image, rate=rate)

It is also possible to inspect the graph using the eagerx-gui package.
It can be installed as follows:
```bash
pip3 install eagerx-gui
```
Jupyter notebooks have limited support for interactive applications, so we cannot open the GUI here.
But if we were to run
```python
graph.gui()
```
The ouput would be as follows:

<img src="./figures/tutorial_1_gui.svg" width=720>

Here we see that the actions of the agent are outputs of *env/actions* and that the observations of the agent are inputs of *env/observations*.
Also, we could render output by connecting to *env/render*, which will be covered in another tutorial.
Note that *env/actions*, *env/observations* and *env/render* represent connections of the `Graph` to the environment.
They are split up in the GUI as nodes for visualization purposes.

Next, we will create the [Environment](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html).
Environment creation in EAGERx follows the same API as Gym, i.e. we have to define a [step()](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html#eagerx.core.env.BaseEnv.step) and [reset()](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html#eagerx.core.env.BaseEnv.reset) method.

In [6]:
from typing import Dict
import numpy as np


class PendulumEnv(eagerx.BaseEnv):
    def __init__(self, name: str, rate: float, graph: eagerx.Graph, engine: eagerx.Engine):
        """Initializes an environment with EAGERx dynamics.

        :param name: The name of the environment. Everything related to this environment
                     (parameters, topics, nodes, etc...) will be registered under namespace: "/[name]".
        :param rate: The rate (Hz) at which the environment will run.
        :param graph: The graph consisting of nodes and objects that describe the environment's dynamics.
        :param engine: The physics engine that will govern the environment's dynamics.
        """
        self.eval = eval
        
        # Maximum episode length
        self.max_steps = 100
        
        # Step counter
        self.steps = None
        super().__init__(name, rate, graph, engine, force_start=True)
    
    def step(self, action: Dict):
        """A method that runs one timestep of the environment's dynamics.

        :params action: A dictionary of actions provided by the agent.
        :returns: A tuple (observation, reward, done, info).

            - observation: Dictionary of observations of the current timestep.

            - reward: amount of reward returned after previous action

            - done: whether the episode has ended, in which case further step() calls will return undefined results

            - info: contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        # Take step
        observation = self._step(action)
        self.steps += 1
        
        # Get angle and angular velocity
        # Take first element because of window size (covered in other tutorial)
        th = observation["angle"][0] 
        thdot = observation["angular_velocity"][0]

        # Convert from numpy array to float
        u = float(action["voltage"])

        # Normalize angle so it lies in [-pi, pi]
        th -= 2 * np.pi * np.floor((th + np.pi) / (2 * np.pi))

        # Calculate cost
        # Penalize angle error, angular velocity and input voltage
        cost = th**2 + 0.1 * thdot**2 + 0.001 * u**2  

        # Determine when is the episode over
        # currently just a timeout after 100 steps
        done = self.steps > self.max_steps

        # Set info, tell the algorithm the termination was due to a timeout
        # (the episode was truncated)
        info = {"TimeLimit.truncated": self.steps > self.max_steps}
        
        return observation, -cost, done, info
    
    def reset(self) -> Dict:
        """Resets the environment to an initial state and returns an initial observation.

        :returns: The initial observation.
        """
        # Determine reset states
        states = self.state_space.sample()
            
        # Perform reset
        observation = self._reset(states)

        # Reset step counter
        self.steps = 0
        return observation

Next, we will create the [Engine](https://eagerx.readthedocs.io/en/master/guide/api_reference/engine/index.html).
Since objects can have implementions for multiple physics engines and real systems, we need to initialize the appropriate engine.
In our case, we will use the [OdeEngine](https://github.com/eager-dev/eagerx_ode), which allows to simulate systems based on ordinary differential equations (ODEs).
In other tutorials we will go more into detail on the engine and how you can create your own engine.

In [7]:
import eagerx_ode  # Registers OdeEngine

# Define engines
engine = eagerx.Engine.make("OdeEngine", rate=rate)

Just like in normal Gym environments, we will create a step function in which we will calculate the reward at each time step and check for termination conditions.
Our goal is to stabilize the pendulum in upright position, while minimizing the input voltage that is applied.
Therefore we choose a reward function that is a weighted sum of $\theta^2$, $\dot{\theta^2}$ and $u^2$. 

Note that we can obtain the values of the actions and observations using the keys *voltage*, *angle* and *angular_velocity*, which correspond to the names of the actions and observations above in the screenshot of the GUI.

Having created a graph, an engine and a step function, we can now construct the EAGERx environment.
We can use it like any other Gym environment.
Here we will now train a policy to swing up the pendulum using the Soft Actor Critic (SAC) reinforcement learning algorithm implementation from [Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/).

In [8]:
import stable_baselines3 as sb3
from eagerx.wrappers import Flatten

# Initialize Environment
env = PendulumEnv(name="PendulumEnv", rate=rate, graph=graph, engine=engine)

# Toggle render
env.render("human")

# Stable Baselines3 expects flattened actions & observations
# Convert observation and action space from Dict() to Box()
env = Flatten(env)

# Initialize learner
model = sb3.SAC("MlpPolicy", env, verbose=1)

# Train for 1 minute (sim time)
model.learn(total_timesteps=int(60 * rate))

env.shutdown()

[INFO] [1653483875.887615]: Node "/PendulumEnv/env/supervisor" initialized.
[INFO] [1653483876.040063]: Node "/PendulumEnv/engine" initialized.
[INFO] [1653483876.207828]: Node "/PendulumEnv/environment" initialized.
[INFO] [1653483876.304769]: Node "/PendulumEnv/pendulum/theta" initialized.
[INFO] [1653483876.346392]: Waiting for nodes "['env/render']" to be initialized.
[INFO] [1653483876.362410]: Node "/PendulumEnv/pendulum/dtheta" initialized.
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[INFO] [1653483876.467694]: Adding object "pendulum" of type "Pendulum" to the simulator.
[INFO] [1653483876.487797]: Node "/PendulumEnv/pendulum/x" initialized.
[INFO] [1653483876.511564]: [pendulum/image] START RENDERING!
[INFO] [1653483876.512159]: Node "/PendulumEnv/pendulum/image" initialized.
[INFO] [1653483876.539035]: Node "/PendulumEnv/pendulum/u" initialized.
[INFO] [1653483876.557002]: Node "/PendulumEnv/pendulum/u_applied" initialized.
[